## NaiveBayes implementation to classify song genre 
### Tasneem Abed (1408535)
#### COMS4030A project 2019

In [ ]:
#Naive Bayes implementation
import numpy as np
from sklearn.model_selection import train_test_split
import csv
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from sklearn.metrics import confusion_matrix

In [ ]:
dataframe = pd.read_csv("FinalNorm.csv")
dataframe.head()

In [ ]:
df = dataframe.copy()
for i in range(0, df.shape[0]):
    df.lyrics.loc[i] = df.lyrics.loc[i].replace('~', ' ')
    df.lyrics.loc[i] = df.lyrics.loc[i].replace("'", '')
df = df.drop(['WPL', 'Unique WPL', 'Token ration', 'Mean word length', 'Total'], axis=1)
df.head()

country = 0<br> pop = 1<br> rap = 2<br> rock = 3

In [ ]:
#Encode genre:
for i in range(0, df.shape[0]):
    if (i>= 0 and i<750):
        df.genre.iloc[i] = 0
    elif (i>= 750 and i<1500):
        df.genre.iloc[i] = 1
    elif (i>= 1500 and i<2250):
        df.genre.iloc[i] = 2
    elif (i>= 2250 and i<3000):
        df.genre.iloc[i] = 3
        

In [ ]:
df.head()

In [ ]:
porter = PorterStemmer()
lancaster=LancasterStemmer()
def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

In [ ]:
#Apply Stemming
for i in range(0, df.shape[0]):
    df.lyrics.iloc[i] = stemSentence(df.lyrics.iloc[i])

In [ ]:
leftover, newtraining = train_test_split(df, test_size=0.7)
newtraining.head()

In [ ]:
newtraining.shape

In [ ]:
y, x = [], []

for row in df.iterrows():
    index, data = row
    k = data[2]
    r = k.split() #Splits the string into individual words by space
    t = list(set(r)) #All the unique words ['aint', 'make', 'the'...]
    g = ' '.join(t) #Takes the list and makes it a single string
    x.append(g)
    y.append(data[3])

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2)

In [ ]:
len(x_test)

In [ ]:
#Checking the balance of the classes
print(y_test.count(0))
print(y_test.count(1))
print(y_test.count(2))
print(y_test.count(3))

In [ ]:
#Get individual lists for each genre
xCo = [x_train[i] for i in range(len(x_train)) if y_train[i] == 0]
xPop = [x_train[i] for i in range(len(x_train)) if y_train[i] == 1]
xRap = [x_train[i] for i in range(len(x_train)) if y_train[i] == 2]
xRock = [x_train[i] for i in range(len(x_train)) if y_train[i] == 3]


In [ ]:
words = []
country_words = []
pop_words = []
rap_words = []
rock_words = []

for i in range(0, len(x_train)):
    h = x_train[i].split()
    for j in range(0, len(h)):
#         if len(h[j]) > 2:   #Remove stop words
        words.append(h[j])

for k in range(0, len(xCo)):
    fo = xCo[k].split()
    for l in range(0, len(fo)):
#         if len(fo[l]) > 2:   #Remove stop words
         country_words.append(fo[l]) 

for m in range(0, len(xPop)):
    fa = xPop[m].split()
    for n in range(0, len(fa)):
#         if len(fa[n]) > 2:   #Remove stop words
        pop_words.append(fa[n]) 
        
for o in range(0, len(xRap)):
    fi = xRap[o].split()
    for p in range(0, len(fi)):
#         if len(fi[p]) > 2:   #Remove stop words
        rap_words.append(fi[p]) 
        
for q in range(len(xRock)):
    fu = xRock[q].split()
    for t in range(len(fu)):
#         if len(fu[t]) > 2:   #Remove stop words
        rock_words.append(fu[t]) 
                
print(len(country_words), len(pop_words), len(rap_words), len(rock_words))

In [ ]:
#Get prior probabilities
priorC = len(xCo)/len(x_train)
priorP = len(xPop)/len(x_train)
priorRap = len(xRap)/len(x_train)
priorRock = len(xRock)/len(x_train)

In [ ]:
country_dict = {}
pop_dict = {}
rap_dict = {}
rock_dict = {}

for i in range(0, len(words)):
    country_dict[words[i]] = (country_words.count(words[i])+1)
    pop_dict[words[i]] = (pop_words.count(words[i])+1)
    rap_dict[words[i]] = (rap_words.count(words[i])+1)
    rock_dict[words[i]] = (rock_words.count(words[i])+1)

In [ ]:
len(x_test)

In [ ]:
#Testing
import math
predictions = []

for i in range(0, len(x_test)):
    print(i)
    testlyric = x_test[i]
    lyric_words = testlyric.split() #Array of separate words from single review    
    c = 0
    po = 0
    ra = 0
    ro = 0
    for key in country_dict.keys():  #P(test lyric | pop)
        if key in lyric_words:
            ct = (country_dict[key] / (len(xCo)+4))
        else:
            ct = 1- (country_dict[key] / (len(xCo)+4))
        c = c+ abs(math.log10(ct))
        
    for key in pop_dict.keys():
        if key in lyric_words:
            pt = (pop_dict[key] / (len(xPop)+4))
        else:
            pt = 1-(pop_dict[key] / (len(xPop)+4))
        po = po+ abs(math.log10(pt))

    for key in rap_dict.keys():
        if key in lyric_words:
            rt = (rap_dict[key] / (len(xRap)+4))
        else:
            rt = 1-(rap_dict[key] / (len(xRap)+4))
        ra = ra + abs(math.log10(rt))
            
    for key in rock_dict.keys():
        if key in lyric_words:
            rot = (rock_dict[key]/ (len(xRock)+4))
        else:
            rot = 1-(rock_dict[key]/ (len(xRock)+4))
        ro = ro + abs(math.log10(rot)) 

    normalization = ((c*priorC)+(po*priorP)+(ra*priorRap)+(ro*priorRock)) 
    NB = []
    NB.append((c*priorC) / normalization) #P(country | lyric)
    NB.append((po*priorP)/ normalization)
    NB.append((ra*priorRap)/ normalization)
    NB.append((ro*priorRock)/ normalization)
    pred = NB.index(min(NB))
#     print(testery[i], pred)
    predictions.append(pred)
#     print(NB)
    
# classification_error_train = np.sum(y_train != predictions) / len(x_train)
classification_error_test = np.sum(y_test != predictions) / len(x_test)


In [ ]:
len(predictions)

In [ ]:
print("Testing error: ", classification_error_test)

In [ ]:
confusion = np.zeros((4,4))
correct = 0
for i in range(0,len(y_test)):
    label = y_test[i]
    prediction = predictions[i]
    if (prediction == label):
        correct += 1
    confusion[prediction, label] += 1
accuracy = (correct/(len(x_test))) * 100
print("Accuracy: %.4f" %(accuracy))
print(confusion)